In [1]:
# default_exp workflow
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload (local) python modules if they are updated
%autoreload 2

In [2]:
# hide
from nbdev.showdoc import *

# Workflow

    Define workflow for automatically updating, training and deploying your ML model!


***input:*** data, model & loss notebooks and related modules

***output:*** script for executing the ML model update workflow

***description:***

A ML model update workflow allows you to automatically reload your data, train, evaluate and deploy your model.
Note that by following the notebook templates you have already done most of the work - the notebooks **are** the workflow!

So, in this notebook you define a script to automatically execute the other notebooks with the [papermill](https://papermill.readthedocs.io/) tool. Note, that you can input parameters to the notebooks!

You can either define static workflow, where every step is always recreated every time,
or a dynamic workflow, where only the parts of the workflow are recreated that are affected by the changes since last model update.
For dynamic workflows we encourage utilizing the [Snakemake](https://snakemake.readthedocs.io/) tool.

Here we present a super simple static workflow example that you can build upon in your project. 

Edit this and other text cells to describe your project. 

Remember that you can utilize the `#export` tag to export cell commands to `[your_module]/workflow.py`.

## Import relevant modules

In [3]:
# export
import papermill
from pathlib import Path
import os

# your code here

## Define notebook parameters

In [4]:
# this cell is tagged with 'parameters'
seed = 0
# your code here

make direct derivations from the paramerters:

In [5]:
# your code here

## Define workflow

Here we present a tiny example you can try running yourself and then extend to your needs.

Note that if you run `nbdev_build_lib`, the script is exported to `[your_module]/workflow.py`.

Then, you can run `python [your_module]/workflow.py` to run the workflow automatically!

In [6]:
# export

"""
A workflow to re-run your machine learning workflow automatically.

This example script will
- rebuild your python module
- run data notebook to reload and clean data
- run model notebook to sw test your model
- run loss notebook to train and evaluate your model with full data,
    and save or deploy it for further use

Feel free to edit!
"""

cwd = Path().cwd()
save_notebooks_to = cwd / "results" / "notebooks"
# Hint! you can also create time or setup -stamped folders to store your results!

# make sure changes are updated to module
# (this will do nothing if you run the workflow from inside a notebook)
os.system("nbdev_build_lib")

# run workflow
for notebook in ["00_data.ipynb", "01_model.ipynb", "02_loss.ipynb"]:
    papermill.execute_notebook(
        notebook,  # this notebook will be executed
        save_notebooks_to
        / ("_" + notebook),  # this is where the executed notebook will be saved
        # (notebooks named with '_' -prefix are ignored by nbdev build_lib & build_docs!)
        parameters={"seed": 1},  # you can change notebook parameters
        kernel_name="python38myenv",
    )  # note: change kernel according to your project setup!

Executing:   0%|          | 0/29 [00:00<?, ?cell/s]

Executing:   0%|          | 0/25 [00:00<?, ?cell/s]

Executing:   0%|          | 0/31 [00:00<?, ?cell/s]

You can also define your workflow in another language than Python and write it into a file from this notebook utilizing the %%writefile magic.
This way your script is still included the documentation without copy-pasting.
You can also add the script to .gitignore to avoid double tracking.

## You can now move on to API notebook!